In [40]:
from scraper import WebScraper
from time import sleep
import pandas as pd
from json import dumps
from tqdm import tqdm
from bs4 import BeautifulSoup
from glob import glob

In [41]:
cp = pd.read_excel('./cp_catalogue.xlsx', dtype=str)
cp.drop_duplicates(subset='Código Postal', inplace=True)
cp['Código Postal'] = cp['Código Postal'].str.zfill(5)
already_scraped = glob('./results/uber_new/*')
already_scraped = [a.split('/')[-1].split('.')[0].split('_')[-1] for a in already_scraped]
cp = cp[~cp['Código Postal'].isin(already_scraped)]

direcciones = [f'{c}, Ciudad de México' for c in cp['Código Postal']]

In [42]:
len(direcciones)

1110

In [43]:
source_cates = dict(
populares_cerca = "https://www.ubereats.com/mx/feeds/TopStoresByCityV2IGSeeAllPlugin?ctx={%22value%22%3A%22DwABDAAAABMLAAEAAAAQD8V1MQpVUWKPUQnIn2I5HQsAAgAAABDULEYWnaJLipP5V7RxL7amCwAEAAAAEK9ufc5REkdYv6YXNgZNQf0ACwABAAAAEBxA7j0TUVxKkQWFgFH5s7ILAAIAAAAQLxUbG3fWQ3Kx7zgfgQRFEwsABAAAABCREWw3l2FPor-fkd50QpsGAAsAAQAAABDLDIKaADdCUqi__1Ys_7jVCwACAAAAEAAAAAAAAAAAAAAAAAAAAAALAAQAAAAQg3yl7qGKRe-6REn5P7ESgAALAAEAAAAQyvJ3c3TWTlm2C1gzqEUmagsAAgAAABDXyceLzvlOI5qfRSu9qBbhCwAEAAAAEOdOgRmc001gjKTb7B6l1s4ACwABAAAAELtCdR2s3VCvqnPBqG4y8vwLAAIAAAAQqFvTuPfJRkGOtNrRPjx8QQsABAAAABCKFpqKWtFEpbctZEJStqJXAAsAAQAAABBGsTwcQ9NecIcUZYnnmaPfCwACAAAAEO0r2Ja7Qk1WgFROdw-pk5ULAAQAAAAQKYBKf0MdQCmH0ZCg2pg4jQALAAEAAAAQbmeItbdXQTWixBKbibxjWgsAAgAAABB0w5T7ySBON57vc1PYnxpPCwAEAAAAEH63wAaO5kiyqPqLnlqeKB4ACwABAAAAEA9kGcrNw02IqBMWl_OACF0LAAIAAAAQbkK-qs3_Rwa8aDgNey1yhAsABAAAABBH8lLwcjxL_Z3-0BQ9lCszAAsAAQAAABDA-RgGUP9E56bY1-ia76lHCwACAAAAEJm1Vc_D10VCl3zxWzaq0ZELAAQAAAAQuUxOhZosTtu30IcgmPSQZwALAAEAAAAQlXigXP7KTEueFNoZeOyBPgsAAgAAABAAAAAAAAAAAAAAAAAAAAAACwAEAAAAEA7dF3GpIU07tSt9GemyAE0ACwABAAAAEPsEnZTG_UqfgqyXVOK7j1YLAAIAAAAQ4aTs8GnWR-aWGidGTWW8IQsABAAAABBdE6nBtApJhrepd2am8TnJAAsAAQAAABDRV7qAknFJG7SeLUaJTtG3CwACAAAAEM8NqMr0e0W0pg5oTQBU8mYLAAQAAAAQEJxt_6nvRDykRYp6AUXkSAALAAEAAAAQY-lxeUZ2SLy3uA8OntnviwsAAgAAABCB3hr0Z1pBs7S2NNBKyea6CwAEAAAAELrcQrTWOkeSv1OEYrTCZ6gACwABAAAAEA6pogAtxkCBgYwfAl6hjn8LAAIAAAAQkA6pQB_-ROuAEpY3KRtgXwsABAAAABBAgwzTwzJHM68jnV5wZWj9AAsAAQAAABCshbhJkYtIWIx3HMNI5Y-SCwACAAAAEJnbnMUGeEojrpGEgTxZWtILAAQAAAAQYYHnj81RSZuhhuVhh1nxTgALAAEAAAAQssdPsbJhRHKoYsNb3SpGDgsAAgAAABAAAAAAAAAAAAAAAAAAAAAACwAEAAAAEBMj4770VkXIuVGFhoo-sREACwABAAAAEAE3fFCFXUuOlq6pQiBf3eoLAAIAAAAQUGd1vFBQRkC_KkxdKVnjLAsABAAAABBUCLp73_dEC5UoCzoCjHzNAAsAAQAAABBPQz-bRD5P05vc9-2cuOQXCwACAAAAEGWji4QvEUybiV1-LNZ7QKYLAAQAAAAQzVA_pqFqRlOvPvWM2swd8QALAAEAAAAQQnswJzxySYWNw4CWnBE4pwsAAgAAABCywijzh6FLdq9PBYU1Y3erCwAEAAAAEJjyg3N7sUaZl6_mFFIn0RUAAA%22%2C%22type%22%3A%22c282c472-346c-43d5-9479-ede33030fe84%22%2C%22format%22%3A%22FORMAT_THRIFT_BYTES%22}&pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMlRsYWxwYW4lMjBDZW50cm8lMjBJJTIyJTJDJTIycmVmZXJlbmNlJTIyJTNBJTIyQ2hJSkg4bjlnSWtBem9VUmNpSWlUWEdoeEk0JTIyJTJDJTIycmVmZXJlbmNlVHlwZSUyMiUzQSUyMmdvb2dsZV9wbGFjZXMlMjIlMkMlMjJsYXRpdHVkZSUyMiUzQTE5LjI4NzE4NjklMkMlMjJsb25naXR1ZGUlMjIlM0EtOTkuMTcwNTI1MyU3RA%3D%3D&ps=1",
)

source_cate = "populares_cerca"

In [44]:
for address in tqdm(direcciones):
    
    scraper = WebScraper()

    scraper.navigate_to_url(source_cates["populares_cerca"])
    
    sleep(1.5)

    # Abrir el menu para gestionar dirección
    scraper.click_element('//*[@id="wrapper"]/header/div/div/div/div/div/div[3]/div[2]/div')

    # Entrar al menu para cambiar dirección
    # scraper.click_element('//*[@id="wrapper"]/div[3]/div/div/div[2]/div[3]/div/div[2]/div[1]/div[2]/a')

    # Ingresar dirección
    final_address = scraper.enter_address(address, 
                                        input_xpath='//*[@id="root"]/div[4]/div/div/div[2]/div/div/div/div/div[2]/div/div[2]/input', 
                                        selection_xpath='//*[@id="root"]/div[4]/div/div/div[2]/div/div/div/div/ul/button[1]'
                                        )
    
    sleep(2)
    cards_data = scraper.scrape_cards('//*[@data-test="feed-desktop"]', 
                                    aditional_info={"cate_source": source_cate,
                                                    "current_address": address,
                                                    "final_address": final_address,
                                                    "platform": 'ubereats'
                                                    }
                                    )
        
    with open(f'./results/uber_new/ubereats_{source_cate}_{address.split(',')[0]}.json', 'w') as f:
        f.write(dumps(cards_data, indent=4))
    
    soup = BeautifulSoup(scraper.driver.page_source, 'html.parser')
    with open(f'./results/uber_new/ubereats_{source_cate}_{address.split(',')[0]}.html', 'w') as f:
        f.write(soup.prettify()) 
    
    scraper.close()
    
    sleep(1)

  0%|          | 0/1110 [00:00<?, ?it/s]2025-11-13 17:35:39,951 - INFO - Navegando a la URL: https://www.ubereats.com/mx/feeds/TopStoresByCityV2IGSeeAllPlugin?ctx={%22value%22%3A%22DwABDAAAABMLAAEAAAAQD8V1MQpVUWKPUQnIn2I5HQsAAgAAABDULEYWnaJLipP5V7RxL7amCwAEAAAAEK9ufc5REkdYv6YXNgZNQf0ACwABAAAAEBxA7j0TUVxKkQWFgFH5s7ILAAIAAAAQLxUbG3fWQ3Kx7zgfgQRFEwsABAAAABCREWw3l2FPor-fkd50QpsGAAsAAQAAABDLDIKaADdCUqi__1Ys_7jVCwACAAAAEAAAAAAAAAAAAAAAAAAAAAALAAQAAAAQg3yl7qGKRe-6REn5P7ESgAALAAEAAAAQyvJ3c3TWTlm2C1gzqEUmagsAAgAAABDXyceLzvlOI5qfRSu9qBbhCwAEAAAAEOdOgRmc001gjKTb7B6l1s4ACwABAAAAELtCdR2s3VCvqnPBqG4y8vwLAAIAAAAQqFvTuPfJRkGOtNrRPjx8QQsABAAAABCKFpqKWtFEpbctZEJStqJXAAsAAQAAABBGsTwcQ9NecIcUZYnnmaPfCwACAAAAEO0r2Ja7Qk1WgFROdw-pk5ULAAQAAAAQKYBKf0MdQCmH0ZCg2pg4jQALAAEAAAAQbmeItbdXQTWixBKbibxjWgsAAgAAABB0w5T7ySBON57vc1PYnxpPCwAEAAAAEH63wAaO5kiyqPqLnlqeKB4ACwABAAAAEA9kGcrNw02IqBMWl_OACF0LAAIAAAAQbkK-qs3_Rwa8aDgNey1yhAsABAAAABBH8lLwcjxL_Z3-0BQ9lCszAAsAAQAAABDA-RgGUP9E56bY1-ia76lHCwACAAAAEJm1Vc_D10VCl3zxWzaq0ZE